<a href="https://colab.research.google.com/github/dangdang2222/skt_ai_fellowship/blob/main/s_dcnn_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import argparse
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import pickle as pk
import tensorflow as tf
import h5py
import gc
import pandas as pd
import multiprocessing
import logging

logging.basicConfig(level=logging.INFO)

In [55]:
CNN_HYPER_PARAMS = {
    'block_size': 4.41106, 
    'conv_activation': 1.69354, 
    'dense_activation': 1.28829, 
    'dilation_rate': 1.839, 
    'dropout': 0.131517, 
    'fc1': 74.786, 
    'fc2': 100, 
    'kernel_size': 0.688211, 
    'l1': 0.000723271, 
    'l2': 0, 
    'learning_rate': 0.001, 
    'nblocks': 4.02792, 
}

BATCH_SIZE = 63
W = 162
EPOCHS = 100


In [56]:
!pip install pyts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import os
import pathlib
import random
import numpy as np 
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from collections import Counter
from pyts.image import GramianAngularField
from pandas import DataFrame
import pandas as pd
import h5py
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle as pk
import gc
import logging

logging.basicConfig(level=logging.INFO)

# deactive panda warning
pd.options.mode.chained_assignment = None

In [58]:
FEATURES = ['alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2',
           'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', 'Fc', 'hs']
files = ['N-CMAPSS_DS01-005.h5','N-CMAPSS_DS02-006.h5']

In [59]:
def load_phm21_data_file(filename):
  with h5py.File(filename, 'r') as hdf:
            # Development set
          W_dev = np.array(hdf.get('W_dev'), dtype=np.float32)             # W
          X_s_dev = np.array(hdf.get('X_s_dev'), dtype=np.float32)         # X_s
          Y_dev = np.array(hdf.get('Y_dev'), dtype=np.float32)             # RUL  
          A_dev = np.array(hdf.get('A_dev'), dtype=np.float32)             # Auxiliary

            # Test set
          W_test = np.array(hdf.get('W_test'), dtype=np.float32)           # W
          X_s_test = np.array(hdf.get('X_s_test'), dtype=np.float32)       # X_s
          Y_test = np.array(hdf.get('Y_test'), dtype=np.float32)           # RUL  
          A_test = np.array(hdf.get('A_test'), dtype=np.float32)           # Auxiliary

            # Varnams
          W_var = np.array(hdf.get('W_var'))
          X_s_var = np.array(hdf.get('X_s_var'))  
          A_var = np.array(hdf.get('A_var'))

            # from np.array to list dtype U4/U5
          W_var = list(np.array(W_var, dtype='U20'))
          X_s_var = list(np.array(X_s_var, dtype='U20'))  
          A_var = list(np.array(A_var, dtype='U20'))

  W = np.concatenate((W_dev, W_test), axis=0)  
  X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
  Y = np.concatenate((Y_dev, Y_test), axis=0)
  A = np.concatenate((A_dev, A_test), axis=0) 

  X = np.concatenate((W, X_s, A, Y), axis=1)
  X = DataFrame(data=X, columns=W_var+X_s_var+A_var+['Y'])
    
  return X

In [60]:
cache_dir = '/content/drive/MyDrive/'
X = load_phm21_data_file(os.path.join(cache_dir, files[0]))

In [61]:
X['id'] = X.unit
X

,alt,Mach,TRA,T2,T24,T30,T48,T50,P15,P2,...,P50,Nf,Nc,Wf,unit,cycle,Fc,hs,Y,id
0,3013.0,0.376362,70.311996,522.314758,618.288574,1470.469849,1849.620728,1269.275635,19.432070,14.484612,...,15.974771,2142.253418,8693.176758,4.621622,1.0,1.0,1.0,1.0,99.0,1.0
1,3020.0,0.376866,70.311996,522.327148,618.296326,1470.415649,1849.519897,1269.177124,19.431385,14.484683,...,15.970518,2142.218506,8693.000000,4.620561,1.0,1.0,1.0,1.0,99.0,1.0
2,3025.0,0.377685,70.311996,522.371826,618.336487,1470.453857,1849.566162,1269.167358,19.435163,14.488223,...,15.969419,2142.258057,8693.106445,4.621065,1.0,1.0,1.0,1.0,99.0,1.0
3,3035.0,0.376992,70.399887,522.282410,618.302185,1470.650879,1850.195068,1269.518677,19.426003,14.477633,...,15.967762,2142.794678,8693.680664,4.624305,1.0,1.0,1.0,1.0,99.0,1.0
4,3043.0,0.377622,70.399887,522.300598,618.345215,1470.640381,1849.950928,1269.254028,19.427485,14.478113,...,15.962570,2143.015137,8693.714844,4.622768,1.0,1.0,1.0,1.0,99.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7641863,3018.0,0.310653,26.367001,517.710449,569.171326,1262.152710,1505.220703,1142.358643,16.327415,14.032529,...,14.255220,1596.571045,7985.475098,2.174156,10.0,82.0,3.0,0.0,0.0,10.0
7641864,3014.0,0.309960,26.367001,517.681885,569.152100,1262.138184,1505.152344,1142.334229,16.325787,14.030422,...,14.256880,1596.577515,7985.456055,2.173944,10.0,82.0,3.0,0.0,0.0,10.0
7641865,3010.0,0.310149,26.367001,517.707214,569.173279,1262.180298,1505.292358,1142.446045,16.329035,14.033521,...,14.259193,1596.552612,7985.560547,2.174648,10.0,82.0,3.0,0.0,0.0,10.0
7641866,3005.0,0.309834,26.367001,517.706116,569.178345,1262.202026,1505.274658,1142.442261,16.330290,14.034248,...,14.261579,1596.586792,7985.651855,2.174786,10.0,82.0,3.0,0.0,0.0,10.0


In [62]:
for file in files[1:]:  
      aux = load_phm21_data_file(os.path.join(cache_dir,  file))
      aux['id'] = aux.unit + X.id.max()
      X = pd.concat((X, aux), axis=0)

      del aux
      gc.collect()

In [63]:
X

,alt,Mach,TRA,T2,T24,T30,T48,T50,P15,P2,...,P50,Nf,Nc,Wf,unit,cycle,Fc,hs,Y,id
0,3013.0,0.376362,70.311996,522.314758,618.288574,1470.469849,1849.620728,1269.275635,19.432070,14.484612,...,15.974771,2142.253418,8693.176758,4.621622,1.0,1.0,1.0,1.0,99.0,1.0
1,3020.0,0.376866,70.311996,522.327148,618.296326,1470.415649,1849.519897,1269.177124,19.431385,14.484683,...,15.970518,2142.218506,8693.000000,4.620561,1.0,1.0,1.0,1.0,99.0,1.0
2,3025.0,0.377685,70.311996,522.371826,618.336487,1470.453857,1849.566162,1269.167358,19.435163,14.488223,...,15.969419,2142.258057,8693.106445,4.621065,1.0,1.0,1.0,1.0,99.0,1.0
3,3035.0,0.376992,70.399887,522.282410,618.302185,1470.650879,1850.195068,1269.518677,19.426003,14.477633,...,15.967762,2142.794678,8693.680664,4.624305,1.0,1.0,1.0,1.0,99.0,1.0
4,3043.0,0.377622,70.399887,522.300598,618.345215,1470.640381,1849.950928,1269.254028,19.427485,14.478113,...,15.962570,2143.015137,8693.714844,4.622768,1.0,1.0,1.0,1.0,99.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6517185,10027.0,0.382473,25.136539,497.040863,544.098145,1203.357910,1421.337891,1077.482788,12.837452,11.143805,...,10.930024,1545.901733,7793.390625,1.597471,15.0,67.0,2.0,0.0,0.0,25.0
6517186,10022.0,0.382158,25.136539,497.035828,544.035034,1203.472412,1422.772461,1078.697754,12.835965,11.144114,...,10.935884,1545.145020,7793.313477,1.601279,15.0,67.0,2.0,0.0,0.0,25.0
6517187,10016.0,0.381969,25.136539,497.043976,544.055786,1203.507690,1422.445068,1078.410034,12.838463,11.145616,...,10.937328,1545.342407,7793.574219,1.600758,15.0,67.0,2.0,0.0,0.0,25.0
6517188,10010.0,0.381717,25.136539,497.047241,544.060120,1203.529907,1422.576050,1078.539307,12.839843,11.146680,...,10.939893,1545.300049,7793.618164,1.601250,15.0,67.0,2.0,0.0,0.0,25.0


In [64]:
features= ['alt', 'Mach', 'TRA', 'T2', 'T24', 'T30', 'T48', 'T50', 'P15', 'P2',
                   'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf', 'Fc', 'hs']


In [65]:
seed = 256
train_set_file_path = os.path.join(cache_dir, 'train_l1_%d.h5' % seed)
test_set_file_path = os.path.join(cache_dir, 'test_l1_%d.h5' % seed)
scaler_file_path = os.path.join(cache_dir, 'scaler_%d.pk' % seed)

In [66]:
def train_test_split(X, train_ratio=0.7, random_seed=256):
    random.seed(random_seed)
    ids = set(X.id.unique())
    train_ids = random.sample(ids, int(len(ids) * train_ratio))
    test_ids =  list(ids - set(train_ids))
    logging.info("Train ids:" + str(train_ids))
    logging.info("Test ids:" + str(test_ids))
    return X[X.id.isin(train_ids)].copy(), X[X.id.isin(test_ids)].copy()

In [67]:
X_train, X_test = train_test_split(X, random_seed=seed)

INFO:root:Train ids:[25.0, 10.0, 21.0, 15.0, 7.0, 20.0, 1.0, 8.0, 24.0, 4.0, 9.0, 3.0, 30.0]
INFO:root:Test ids:[2.0, 5.0, 6.0, 12.0, 26.0, 28.0]


In [68]:
def normalize_data(X_train, X_test, features):
    
    #scaler = MinMaxScaler()
    scaler = StandardScaler()
    values = scaler.fit_transform(X_train[features].values).round(3)
        
    for i,feature in enumerate(features):
        X_train[feature] = values[:, i]

    values = scaler.transform(X_test[features].values).round(3)
    
    for i,feature in enumerate(features):
        X_test[feature] = values[:, i]

    return X_train, X_test, scaler

In [69]:
X_train, X_test, scaler = normalize_data(X_train, X_test, features)

In [70]:
pk.dump(scaler, open(scaler_file_path, 'wb'))
X_train.to_hdf(train_set_file_path, key='phm21')
X_test.to_hdf(test_set_file_path, key='phm21')

del X_train
del X_test
                #del train_gen
                #del text_gen
gc.collect()

34

In [71]:
model_path = os.path.join(cache_dir, 'cnn_l1_%d.h5' % seed)

In [72]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation, MaxPooling2D, Reshape, Input, Dropout
from tensorflow.keras.layers import BatchNormalization, Lambda, Conv2DTranspose, Add
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import constraints
import tensorflow.keras.backend as K
import numpy as np
import inspect
from tensorflow.keras.layers.experimental.preprocessing import Resizing
    
activations = ['relu', tf.keras.layers.LeakyReLU(alpha=0.1), 'tanh']
kernels = [(3,3), (10, 1), (10, 5)]      
scorers = [None, 'mean_squared_error']

In [95]:
import tensorflow as tf

class NASAScore(tf.keras.metrics.Metric):

    def __init__(self, name='NASA_score', **kwargs):
        super(NASAScore, self).__init__(name=name, **kwargs)
        self.scores = self.add_weight(name='sc', initializer='zeros')
        self.total_samples = self.add_weight(name='ts', initializer='zeros', dtype=tf.int32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.total_samples.assign_add(tf.shape(y_true)[0])

        #residual = y_true - tf.math.maximum(y_pred, [0])
        residual = y_true - y_pred
        ueb = tf.cast(tf.less_equal(residual, [0]), tf.float32)
        uev = tf.math.exp( (1/13) * tf.math.abs(residual * ueb) ) -1
        ovv = tf.math.exp( (1/10) * tf.math.abs(residual * (1 - ueb)) ) -1

        self.scores.assign_add(tf.reduce_sum(uev) + tf.reduce_sum(ovv))

    def result(self):
        return self.scores / tf.cast(self.total_samples, tf.float32)

    def reset_state(self):
        self.scores.assign(0)
        self.total_samples.assign(0)
        
class PHM21Score(tf.keras.metrics.Metric):

    def __init__(self, name='Score', **kwargs):
        super(PHM21Score, self).__init__(name=name, **kwargs)
        self.nasa_score = NASAScore()
        self.rmse = tf.keras.metrics.RootMeanSquaredError()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.nasa_score.update_state(y_true, y_pred, sample_weight)
        self.rmse.update_state(y_true, y_pred, sample_weight)

    def result(self):
        return 0.5 * self.nasa_score.result() + 0.5 * self.rmse.result()

    def reset_state(self):
        self.nasa_score.reset_state()
        self.rmse.reset_state()

In [96]:
def create_cnn_model(input_shape, block_size=2, nblocks=2, l1=1e-5, l2=1e-4, 
                     kernel_size=0, dropout=0.5, learning_rate=1e-3, fc1=256, fc2=128,
                     conv_activation=2, dense_activation=2, dilation_rate=1,
                     batch_normalization=1, scorer=1):
    block_size = int(round(block_size))
    nblocks = int(round(nblocks))
    scorer = scorers[int(round(scorer))]
    fc1 = int(round(fc1))
    fc2 = int(round(fc2))
    dilation_rate = int(round(dilation_rate))
    conv_activation = activations[int(round(conv_activation))]
    dense_activation = activations[int(round(dense_activation))]
    kernel_size = kernels[int(round(kernel_size))]
    batch_normalization = True if batch_normalization == 1 else False
    
    input_tensor = Input(input_shape)
    x = input_tensor
    for i, n_cnn in enumerate([block_size] * nblocks):
        for j in range(n_cnn):
            x = Conv2D(32*2**min(i, 2), kernel_size=kernel_size, padding='same', 
                       kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                       kernel_initializer='he_uniform',
                       dilation_rate=dilation_rate,
                       name='Conv%d_Block%d' % (j, i) )(x)
            if batch_normalization:
                x = BatchNormalization(name='BN%d_Block%d' % (j, i))(x)
            x = Activation(conv_activation, name='A%d_Block%d' % (j, i))(x)
        x = MaxPooling2D(2, name='MP%d_Block%d' % (j, i))(x)
        if dropout > 0:
            x = Dropout(dropout, name='DO%d_Block%d' % (j, i))(x)

    x = Flatten()(x)
    
    # FNN
    x = Dense(fc1, name='Fc1', 
             kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(x)
    x = Activation(dense_activation, name='Act_Fc1', )(x)
    if dropout > 0:
        x = Dropout(dropout, name='DO_Fc1')(x)
    x = Dense(fc2, name='Fc2', 
             kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2))(x)
    x = Activation(dense_activation, name='Act_Fc2')(x)
    if dropout > 0:
        x = Dropout(dropout, name='DO_Fc2')(x)
    x = Dense(1, activation='relu', name='predictions')(x) 
    model = Model(inputs=input_tensor, outputs=x)
    model.compile(loss=scorer, optimizer=Adam(learning_rate=learning_rate), 
                  metrics=[NASAScore(), PHM21Score(), tf.keras.metrics.MeanAbsoluteError(name="MAE")])
    
    return model

In [97]:
def describe_features(X, features):
    d = str(X[features].describe().T[['min', 'max', 'mean', 'std']]).split('\n')
    for l in d:
        logging.info(l)


In [98]:
class DataGenerator(Sequence):
    """
    """
    def __init__(self, X, attributes, window_size=10, batch_size=32, 
                 noise_level=0, epoch_len_reducer=100, add_extra_channel=False,
                 return_label=True):
        self.batch_size = batch_size
        self.return_label = return_label
        self.window_size = window_size
        self.noise_level = noise_level
        self.attributes = attributes
        self.epoch_len_reducer = epoch_len_reducer
        self._X = {}
        self._Y = {}
        self._ids = X.id.unique()
        self.add_extra_channel = add_extra_channel
        for _id in self._ids:
            self._X[_id] = X.loc[(X.id==_id), self.attributes].values
            self._Y[_id] = X.loc[(X.id==_id), 'Y'].values
        self.__len = int((X.groupby('id').size() - self.window_size).sum() / 
                        self.batch_size)
        del X


    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return int(self.__len / self.epoch_len_reducer)
    
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        X = self._X
        _X = []
        _y = []
        for _ in range(self.batch_size):
            sid = random.choice(self._ids)
            unit = self._X[sid]
            nrows = unit.shape[0]
            cut = random.randint(0, nrows - self.window_size)
            s = unit[cut: cut + self.window_size].T
            y =self._Y[sid][cut + self.window_size-1]
            _X.append(s)
            _y.append(y)

        
        _X = np.array(_X)
        if self.add_extra_channel:
            _X = _X.reshape(_X.shape + (1,))
            
        if self.noise_level > 0:
            noise_level = self.noise_level
            noise = np.random.normal(-noise_level, noise_level, _X.shape)
            _X = _X + noise
            _X = (_X - _X.min()) / (_X.max() - _X.min())
       
        if self.return_label:
            return _X, np.array(_y).reshape((self.batch_size, 1))
        else:
            return _X, _X
        
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        pass

In [99]:
from tensorflow.keras.callbacks import Callback
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
from collections import defaultdict
import logging

logging.basicConfig(level=logging.INFO)

In [100]:
class PlotL1RUL(tf.keras.callbacks.Callback):
    
    def __init__(self, X, Y, W):
        self.X = X
        self.Y = Y
        self.W = W
        
        self.__fig, axs = plt.subplots(2, 2, figsize=(20, 10))
        self._axs = [ax for aux in axs for ax in aux ]
        
        self.__plotted = False
        self.__count = 0

    def on_train_batch_end(self, batch, logs=None):
        if self.__count != 0:
            for ax in self._axs:
                ax.cla()
        
        self.__count += 1
        self.__fig.suptitle('4 random units from validation set')
        
        if (self.__count % 20) == 0:
            W = self.W

            predictions = defaultdict(lambda : {'y': [], 'p': []})
            for _id in list(self.X.keys()):
                batch = []
                y = []
                unit = self.X[_id]
                for c in range (0, unit.shape[0]-W, 5000):
                    batch.append(unit[c: c + W].T)
                    y.append(self.Y[_id][c + W])

                batch = np.array(batch)
                batch = batch.reshape(batch.shape + (1,))

                p = self.model.predict(batch, batch_size = 256, verbose=False)

                predictions[_id]['y'] = y
                predictions[_id]['p'] = p.reshape((p.shape[0],))

            for i,u in enumerate(predictions.keys()):
                self._axs[i].plot(predictions[u]['y'])
                self._axs[i].plot(predictions[u]['p'])
                self._axs[i].set_xticks([])

            self.__fig.savefig('train_progress.png', format='png')    


In [101]:
def train(seed, W, batch_size, epochs, hyper_params, cache_dir, net_summary=True): 
  
        # crate the model
        model = create_cnn_model((20,W,1), **hyper_params)
        model.summary()
        
        # read fold data sets
        train_set_file_path = os.path.join(cache_dir, 'train_l1_%d.h5' % seed)
        test_set_file_path = os.path.join(cache_dir, 'test_l1_%d.h5' % seed)
        
        
        # create data generators
        extra_channel = True
        X_train = pd.read_hdf(train_set_file_path, key='phm21')
        logging.info("Feature stats of the train set:")
        describe_features(X_train, FEATURES)
        train_gen = DataGenerator(X_train, FEATURES, window_size=W, 
                                      batch_size=batch_size, 
                                      epoch_len_reducer=1000, 
                                      add_extra_channel=extra_channel)
        del X_train
        
        X_test = pd.read_hdf(test_set_file_path, key='phm21')
        logging.info("\n\n---------------------------------------------------------------------")
        logging.info("Feature stats of the test set:")
        describe_features(X_test, FEATURES)
        test_gen = DataGenerator(X_test, FEATURES, window_size=W, 
                                     batch_size=256, epoch_len_reducer=1000, 
                                     add_extra_channel=extra_channel)
        del X_test

        
        train_gen.epoch_len_reducer = 2000
        train_gen.return_label = True
        
        test_gen.epoch_len_reducer = 400
        test_gen.return_label = True

        # plot rul data
        ids = list(test_gen._X.keys())[:4]
        X_rul = {_id:test_gen._X[_id] for _id in ids}
        Y_rul = {_id:test_gen._Y[_id] for _id in ids}
        
        # train
        es = tf.keras.callbacks.EarlyStopping(monitor='val_Score', patience=8)
        rlr = tf.keras.callbacks.ReduceLROnPlateau(patience=3)
        pr = PlotL1RUL(X_rul, Y_rul, W)
        history = model.fit(train_gen, validation_data=test_gen, 
                            epochs=epochs, verbose=2, 
                            callbacks=[es, rlr, pr])
        
        history_path = os.path.join(cache_dir, 'cnn_history_%d.pk' % seed)
        pk.dump(history.history, open(history_path, 'wb'))
        model_path = os.path.join(cache_dir, 'cnn_l1_%d.h5' % seed)
        model.save(model_path)

In [ ]:
if not os.path.exists(model_path):
          queue = multiprocessing.Queue()
          p = multiprocessing.Process(target=train, args=(seed, W, BATCH_SIZE, EPOCHS, CNN_HYPER_PARAMS, 
                                                            cache_dir, seed==256))
          p.start()
          p.join()